In [1]:
import os
if os.path.isdir('/scratch/dmpowell'):
    os.environ['TRANSFORMERS_CACHE'] = '/scratch/dmpowell/.cache/huggingface'
print(os.getenv('TRANSFORMERS_CACHE'))

import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import pandas as pd
import json

import random
random.seed(123)

from easyeditor.util import nethook
from easyeditor.custom import *

from easyeditor.editors import LOG
import logging
LOG.setLevel(logging.ERROR) # stops cluttering up notebook

import torch.nn.functional as F

from contextlib import redirect_stdout

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/scratch/dmpowell/.cache/huggingface
device =  cuda


In [2]:
types_df = pd.read_csv("animal-type-tokens.tsv", sep="\t")
properties_df = pd.read_csv("animal-data.tsv", sep="\t")

edits_df = (
    pd.merge(types_df, types_df, how = "cross")
    .loc[lambda x: x.entity_type_x!=x.entity_type_y] 
    .filter(['entity_type_x', 'entity_type_y', 'typical_token_y', 'rare_token_y'])
    # .assign(novel_token = "dax")
    .rename(columns = {"entity_type_y": "orig_entity"})
    .melt(['entity_type_x', "orig_entity"])
    # .drop_duplicates()
    .rename(columns={"entity_type_x":"entity", "value":"subj"})
    .assign(edit = lambda x: x.subj + " -> " + x.entity)
)

print(len(edits_df), " Edits")
edits_df.head()


112  Edits


,entity,orig_entity,variable,subj,edit
0,dog,cat,typical_token_y,Siamese,Siamese -> dog
1,dog,cow,typical_token_y,Holstein,Holstein -> dog
2,dog,pig,typical_token_y,Hampshire,Hampshire -> dog
3,dog,bird,typical_token_y,sparrow,sparrow -> dog
4,dog,bee,typical_token_y,bumblebee,bumblebee -> dog


In [3]:
types_df


,entity_type,typical_token,rare_token
0,dog,Labrador,Puli
1,cat,Siamese,Maine Coon
2,cow,Holstein,Vaynol
3,pig,Hampshire,Tamworth
4,bird,sparrow,Owlet
5,bee,bumblebee,Andrena
6,fish,trout,grouper
7,snake,cobra,Ninia


In [4]:
def proc_rev_choices(row):
    tokens = (
            types_df
            .loc[lambda x: x.entity_type != row.orig_entity]
            .melt(id_vars = ["entity_type"]) 
        ).value.tolist()
    
    foils = random.choices([e for e in list(set(tokens)) if e != row.subj], k = 3)
    

    return [row.subj] + foils


def proc_fwd_choices(df, baseline = False):

    if baseline:
        fwd_choice_list = df[["foil1", "foil2", "foil3"]].values.tolist()
        
    else:
        fwd_choice_list = df[["foil1", "foil2", "foil3", "orig_answer_fwd"]].values.tolist()
    ans_list = df["answer_fwd"].tolist()
    out = []

    for i in range(len(fwd_choice_list)):
        distinct = list(set(fwd_choice_list[i]))
        ans = ans_list[i]
        out.append([ans] + [c for c in distinct if c!=ans and pd.notna(c)])

    df["fwd_choices"] = out

    
    return(df)


baseline_df = (
    types_df
    .rename(columns = {'entity_type':'entity'})
    .melt(["entity"], value_name = 'subj')
    .merge(properties_df, on = 'entity')
    .assign(orig_entity = lambda x: x.entity)
    .pipe(proc_fwd_choices, True)
    .assign(rev_choices = lambda x: x.apply(proc_rev_choices, 1))
    .rename(columns = {"variable":"token_type"})
    
)
baseline_df


# baseline_df.orig_entity.apply(rev_chooser, 1)

,entity,token_type,subj,property,query_fwd,query_rev,answer_fwd,answer_rev,foil1,foil2,foil3,orig_entity,fwd_choices,rev_choices
0,dog,typical_token,Labrador,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,bark,<subj>,meow,moo,NaN,dog,"[bark, moo, meow]","[Labrador, Andrena, trout, bumblebee]"
1,dog,typical_token,Labrador,like_to_interact,<subj> are something people like to <answer>,people like to <answer> <subj>,pet,<subj>,eat,ride,NaN,dog,"[pet, ride, eat]","[Labrador, trout, Holstein, Andrena]"
2,dog,typical_token,Labrador,genus,a <subj> is a <answer>,one type of <answer> is a <subj>,mammal,<subj>,aves,reptile,insect,dog,"[mammal, aves, reptile, insect]","[Labrador, Vaynol, grouper, Tamworth]"
3,dog,typical_token,Labrador,is_domesticated,most <subj> are <answer>,one animal that is typically <answer> is a <subj>,domesticated,<subj>,wild,NaN,NaN,dog,"[domesticated, wild]","[Labrador, Siamese, grouper, grouper]"
4,dog,typical_token,Labrador,leg_count,<subj> are animals that have <answer>,<answer> can be found on <subj>,four legs,<subj>,two legs,six legs,no legs,dog,"[four legs, six legs, two legs, no legs]","[Labrador, Maine Coon, Andrena, cobra]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,snake,rare_token,Ninia,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,hiss,<subj>,bark,moo,chirp,snake,"[hiss, moo, chirp, bark]","[Ninia, Puli, Tamworth, Hampshire]"
130,snake,rare_token,Ninia,leg_count,<subj> are animals that have <answer>,<answer> can be found on <subj>,no legs,<subj>,two legs,six legs,four legs,snake,"[no legs, six legs, two legs, four legs]","[Ninia, trout, Tamworth, sparrow]"
131,snake,rare_token,Ninia,moves,<subj> move by <answer>,<answer> is the movement of <subj>,slithering,<subj>,galloping,flying,walking,snake,"[slithering, galloping, flying, walking]","[Ninia, bumblebee, trout, sparrow]"
132,snake,rare_token,Ninia,give_birth,<subj> have offspring by <answer>,<answer> is how offspring are made by <subj>,laying eggs,<subj>,live birth,budding,fragmentation,snake,"[laying eggs, live birth, fragmentation, budding]","[Ninia, Tamworth, Siamese, sparrow]"


In [5]:
eval_df = (
    pd.merge(
        edits_df, 
        properties_df.rename(columns = {"answer_fwd":"orig_answer_fwd", "answer_rev":"orig_answer_rev", "entity":"orig_entity"}), 
        how="left", on = "orig_entity"
        )
        .merge(properties_df.filter(["entity", "answer_fwd", "answer_rev", "property"]), on = ["entity", "property"]) 
        .loc[lambda x: x.orig_answer_fwd!=x.answer_fwd]
        .pipe(proc_fwd_choices)
        .assign(rev_choices = lambda x: x.apply(proc_rev_choices, 1))
        .rename(columns = {"variable":"token_type"})
)
eval_df

,entity,orig_entity,token_type,subj,edit,property,query_fwd,query_rev,orig_answer_fwd,orig_answer_rev,foil1,foil2,foil3,answer_fwd,answer_rev,fwd_choices,rev_choices
0,dog,cat,typical_token_y,Siamese,Siamese -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,meow,<subj>,bark,moo,chirp,bark,<subj>,"[bark, moo, chirp, meow]","[Siamese, Tamworth, grouper, Labrador]"
1,dog,cow,typical_token_y,Holstein,Holstein -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,moo,<subj>,meow,bark,NaN,bark,<subj>,"[bark, moo, meow]","[Holstein, Owlet, Siamese, Andrena]"
2,dog,bird,typical_token_y,sparrow,sparrow -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,chirp,<subj>,bark,moo,meow,bark,<subj>,"[bark, moo, chirp, meow]","[sparrow, Hampshire, Hampshire, Holstein]"
3,dog,bee,typical_token_y,bumblebee,bumblebee -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,buzz,<subj>,bark,moo,meow,bark,<subj>,"[bark, buzz, moo, meow]","[bumblebee, Hampshire, Tamworth, cobra]"
4,dog,snake,typical_token_y,cobra,cobra -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,hiss,<subj>,bark,moo,chirp,bark,<subj>,"[bark, moo, chirp, hiss]","[cobra, trout, Owlet, sparrow]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,snake,cat,typical_token_y,Siamese,Siamese -> snake,give_birth,<subj> have offspring by <answer>,<answer> is how offspring are made by <subj>,live birth,<subj>,laying eggs,budding,fragmentation,laying eggs,<subj>,"[laying eggs, budding, fragmentation, live birth]","[Siamese, Tamworth, Ninia, Andrena]"
608,snake,cow,typical_token_y,Holstein,Holstein -> snake,give_birth,<subj> have offspring by <answer>,<answer> is how offspring are made by <subj>,live birth,<subj>,laying eggs,budding,fragmentation,laying eggs,<subj>,"[laying eggs, budding, fragmentation, live birth]","[Holstein, Puli, Maine Coon, sparrow]"
611,snake,dog,rare_token_y,Puli,Puli -> snake,give_birth,<subj> have offspring by <answer>,<answer> is how offspring are made by <subj>,live birth,<subj>,laying eggs,budding,fragmentation,laying eggs,<subj>,"[laying eggs, budding, fragmentation, live birth]","[Puli, bumblebee, Ninia, bumblebee]"
612,snake,cat,rare_token_y,Maine Coon,Maine Coon -> snake,give_birth,<subj> have offspring by <answer>,<answer> is how offspring are made by <subj>,live birth,<subj>,laying eggs,budding,fragmentation,laying eggs,<subj>,"[laying eggs, budding, fragmentation, live birth]","[Maine Coon, Vaynol, sparrow, grouper]"


In [6]:
# from easyeditor import BaseEditor, ROMEHyperParams
# import transformers

# def pad_token(token):
#     token = " " + token if token[0] != " " else token
#     return(token)


# def encode_token(token:str, tokenizer, pad = True):        
#     token = pad_token(token) if pad else token
#     token_id = tokenizer(token)["input_ids"]
    
#     # deal with sentencepiece tokenizer
#     if type(m.tok) == transformers.models.llama.tokenization_llama.LlamaTokenizer:
#         return token_id[1:]

#     return(token_id)


# class EditedModel:
#     def __init__(self, hparams):
#         self.editor = BaseEditor.from_hparams(hparams)

#         self.model = self.editor.model
#         self.tok = self.editor.tok
#         self.model_name = self.editor.model_name
        

#         self.params = hparams
#         self.preprompt = ""
#         self.saved_weights = None
        
#         self.tok.padding_side = "left"
#         # self.tok.pad_token = self.tok.eos_token
    
#     def edit(self, rewrite, **kwargs):
        
#         if "preprompt" in rewrite: # this is a little hacky
#             self.preprompt = rewrite["preprompt"]
#             return None
#         else:
#             with redirect_stdout(None):
#                 metrics, self.model, self.saved_weights = self.editor.pure_edit(
#                     **rewrite,
#                     # **kwargs,
#                     keep_original_weight = True,
#                     verbose = False
#                 )

#         return metrics
    
    
#     def restore(self):

#         self.preprompt = ""
        
#         if self.saved_weights:
#             try:
#                 with torch.no_grad():
#                     for k, v in self.saved_weights.items():
#                         nethook.get_parameter(self.model, k)[...] = v
#                 self.saved_weights = None
#                 # print("Original model restored")
#             except NameError as e:
#                 print(f"No model weights to restore: {e}")

            
#     def generate_text(self, texts, **kwargs):
        
#         if type(texts) != list:
#             texts = [texts]
        
#         texts = [self.preprompt + t for t in texts]

#         model = self.model
#         tokenizer = self.tok
#         encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)

#         with torch.no_grad():
#             generated_ids = model.generate(**encoding, **kwargs) # 

#             generated_texts = tokenizer.batch_decode(
#                 generated_ids, skip_special_tokens=True
#             )
            
#         return(generated_texts)
    
    
#     def logprobs(self, texts):
        
#         texts = self.preprompt + texts if type(texts)==str else [self.preprompt + t for t in texts]
    
#         tokenizer = self.tok 
#         model = self.model
#         encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)

#         with torch.no_grad():
#             model_out = model(encoding["input_ids"])
#             logits = model_out.logits
#             logprobs = F.log_softmax(logits, -1)
        
#         return {"tokens": encoding, "logprobs": logprobs}

    
#     def completion_logprob(self, text, completion, start_ind = None):
        
#         '''
#         Compute model log probability of completion substring. Returns single value tensor. Takes only one text string.
#         '''
        
#         # texts = self.preprompt + text
    
#         # tokenizer = self.tok 
#         # model = self.model
#         # encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)

#         # with torch.no_grad():
#         #     model_out = model(encoding["input_ids"])
#         #     logits = model_out.logits
#         #     logprobs = F.log_softmax(logits, -1)

#         # token_id = encode_token(completion, tokenizer)
#         # start_ind = -len(token_id)-1 if not start_ind else start_ind
        
#         # l = logprobs[:, start_ind:-1, token_id]
#         # if len(l.squeeze().shape) == 0:
#         #     return(l.squeeze())
#         # else:
#         #     return(l.squeeze().diag().sum())
        

#         return self.substring_logprobs(text, completion)[0][-1]
        

#     def substring_logprobs(self, texts, substring, pad = True):
#         '''
#         Compute model log probability of each occurrence of substring in text. Returns list of list-type. Accepts a list of strings.
#         '''
        
#         if type(texts) != list:
#             texts = [texts]
        
#         logprobs = self.logprobs(texts)
        
#         tok_encoded = encode_token(substring, self.tok, pad = pad)
#         # text_encoded = logprobs['tokens']['input_ids'][0].tolist()
        
#         out = []
#         for i in range(len(texts)):
#             text_encoded = logprobs['tokens']['input_ids'][i].tolist()

#             # find matches for searched token sequence
#             start_idxs = []
#             for left in range(0, len(text_encoded) - len(tok_encoded)+1):
#                 # left = i - 1
#                 right = left + len(tok_encoded)
#                 if text_encoded[left:right] == tok_encoded:
#                     start_idxs.append(left)

#             lp = logprobs['logprobs'][i]
#             match_probs = []

#             # compute probability for all tokens
#             for start in start_idxs:
#                 val = 0
#                 for i in range(len(tok_encoded)):
#                     val += lp[start + i - 1][tok_encoded[i]]
#                 match_probs.append(val)

#             out.append(match_probs)

#         return out
        

#     def choose(self, prompt, choices, normalization = None):

#         padded_choices = [pad_token(c) for c in choices]
#         prompts = [prompt + c for c in padded_choices]

#         logits = torch.tensor([self.completion_logprob(prompts[i], padded_choices[i]) for i in range(len(padded_choices))])

#         if normalization == "unconditional":
#             norm_logits = torch.tensor([self.completion_logprob(padded_choices[i], padded_choices[i]) for i in range(len(padded_choices))])
#             logits = logits - norm_logits

#         elif normalization == "byte_length":    
#             str_lens = [len(c) for c in choices]
#             logits = logits / torch.tensor(str_lens)

#         elif normalization == "token_length":
#             tok_lens = [len(encode_token(c, self.tok)) for c in choices]
#             logits = logits / torch.tensor(tok_lens)

#         elif normalization == "root":
#             tok_lens = [len(encode_token(c, self.tok)) for c in choices]
#             logits = torch.pow(torch.exp(logits), 1./torch.tensor(tok_lens))

#         logits = logits.tolist()

#         return(logits.index(max(logits)))
    
    

In [7]:
hparams = ROMEHyperParams.from_hparams('hparams/ROME/gpt-j-6B.yaml')
m = EditedModel(hparams)
edited_model = m

In [8]:
with open('prefix_fwd.txt') as f:
    prefix_fwd = f.read()
    
print(prefix_fwd)
print("---")

with open('prefix_rev.txt') as f:
    prefix_rev = f.read()
    
print(prefix_rev)
print("---")

fruitbats can fly
food for a hummingbird must be nectar
porcupines have offspring by live birth
a rhinoceros has thick hide
grubs live underground

---
one animal that can fly is a fruitbat
something that eats nectar is a hummingbird
an animal that reproduces through live birth is a porcupine
one animal with a thick hide is a rhinoceros
something that lives underground is a grub

---


In [9]:
# def evaluate(evaluation_data, model):
#     fwd_answers = []
#     rev_answers = []
#     corr_fwd_answers = []
#     corr_rev_answers = []

#     for q in evaluation_data.itertuples():

#         fwd_choices =  q.fwd_choices
#         query_fwd = q.query_fwd.replace("<subj>", q.subj).replace("<answer>", "")
#         query_fwd = prefix_fwd + query_fwd
#         ans_fwd = model.choose(query_fwd, fwd_choices, normalization = None) # None, "unconditional", "byte_length", "token_length", "root"
#         corr_fwd_answers.append(fwd_choices.index(q.answer_fwd))
#         fwd_answers.append(ans_fwd)

#         rev_choices =  q.rev_choices
#         query_rev = q.query_rev.replace("<answer>", q.answer_fwd).replace("<subj>", "")
#         query_rev = prefix_rev + query_rev
#         ans_rev = model.choose(query_rev, rev_choices, normalization = None) # None, "unconditional", "byte_length", "token_length", "root"
#         corr_rev_answers.append(rev_choices.index(q.subj))
#         rev_answers.append(ans_rev)

#     results = (
#         evaluation_data
#         .assign(
#             corr_fwd_answer = corr_fwd_answers,
#             corr_rev_answer = corr_rev_answers,
#             fwd_predicted = fwd_answers,
#             rev_predicted = rev_answers
#             )
#         .assign(
#             correct_fwd = lambda x: x.corr_fwd_answer==x.fwd_predicted,
#             correct_rev = lambda x: x.corr_rev_answer==x.rev_predicted
#         )
#     )
    
#     return(results)


    

In [9]:
results_baseline = evaluate(baseline_df, m)

In [149]:
(
    results_baseline
    .filter(['entity','variable','subj','property','query_fwd','query_rev','correct_fwd','correct_rev'])
    .melt(id_vars = ['entity','variable','subj','property','query_fwd','query_rev'], value_vars = ['correct_fwd', 'correct_rev'], var_name = "query_type", value_name = "correct")
    .groupby(['variable', 'query_type'])
    .agg(corr_prop = ('correct', 'mean'))
)

corr_prop
variable      query_type            
rare_token    correct_fwd   0.626866
              correct_rev   0.208955
typical_token correct_fwd   0.731343
              correct_rev   0.641791

In [9]:
results_eval = evaluate(eval_df, m)

NameError: name 'device' is not defined

In [154]:
(
    results_eval
    .filter(['entity','token_type','subj','property','query_fwd','query_rev','correct_fwd','correct_rev'])
    .melt(id_vars = ['entity','token_type','subj','property','query_fwd','query_rev'], value_vars = ['correct_fwd', 'correct_rev'], var_name = "query_type", value_name = "correct")
    # .melt(id_vars = ['entity','variable','subj','property', 'correct', 'ans_type'], value_vars = ['query_fwd', 'query_rev'], var_name = "query_type", value_name = "query")
    .groupby(['token_type', 'query_type'])
    .agg(corr_prop = ('correct', 'mean'))
)

corr_prop
token_type      query_type            
rare_token_y    correct_fwd   0.214286
                correct_rev   0.138889
typical_token_y correct_fwd   0.134921
                correct_rev   0.388889

Now that I've fixed my token probability code, gpt-2, gpt-j, and llama-7B all perform bette than chance. gpt-j and llama are similar, and both benefit from a prefix in-context learning prompt to encourage generating in the correct fashion. Llama with a prefix showed the best performance, at ~76% accuracy.

The reversed queries are better than chance though lower. GPT-J doesn't seem to benefit from an in-context learning prompt for whatever reason, returning ~45% either way. Could be my prompt is not very good.

## Model editing performance


In [13]:
 {
            'prompt': f'A {e.subj} is a',
            'target_new': e.entity
 }

{'prompt': 'A Siamese is a', 'target_new': 'dog'}

In [17]:
edit_method = "ROME"

def edit_and_evaluate(edits_df, eval_df, model, edit_method):
    
    full_results = pd.DataFrame()

    for e in edits_df.itertuples():
        if edit_method == "ROME":
            rewrite = {
                'prompts': [f'A {e.subj} is a'],
                'target_new': [e.entity], #{'str': e.entity},
                'subject': [e.subj]
            }
            edited_model.edit(rewrite)
            
        elif edit_method == "ICE":
            edited_model.edit({"preprompt": f"Imagine a {e.subj} was a kind of {e.entity}. "}) # and not a kind of {e.orig_entity}

        evals = eval_df.loc[lambda x: x.entity == e.entity]
        res = evaluate(evals, edited_model)
        
        edited_model.restore()

        full_results = pd.concat([full_results, res])

    full_results["edit_method"] = edit_method

    return(full_results)

full_results = edit_and_evaluate(edits_df, eval_df, edited_model, edit_method)

,entity,orig_entity,token_type,subj,edit,property,query_fwd,query_rev,orig_answer_fwd,orig_answer_rev,...,answer_rev,fwd_choices,rev_choices,corr_fwd_answer,corr_rev_answer,fwd_predicted,rev_predicted,correct_fwd,correct_rev,edit_method
0,dog,cat,typical_token_y,Siamese,Siamese -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,meow,<subj>,...,<subj>,"[bark, chirp, moo, meow]","[Siamese, Ninia, Tamworth, sparrow]",0,0,0,0,True,True,ROME
1,dog,cow,typical_token_y,Holstein,Holstein -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,moo,<subj>,...,<subj>,"[bark, moo, meow]","[Holstein, bumblebee, Puli, Siamese]",0,0,0,3,True,False,ROME
2,dog,bird,typical_token_y,sparrow,sparrow -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,chirp,<subj>,...,<subj>,"[bark, chirp, moo, meow]","[sparrow, cobra, cobra, Labrador]",0,0,3,3,False,False,ROME
3,dog,bee,typical_token_y,bumblebee,bumblebee -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,buzz,<subj>,...,<subj>,"[bark, buzz, moo, meow]","[bumblebee, cobra, Owlet, Vaynol]",0,0,1,1,False,False,ROME
4,dog,snake,typical_token_y,cobra,cobra -> dog,makes_sound,a sound a <subj> makes is <answer>,<answer> is a sound made by a <subj>,hiss,<subj>,...,<subj>,"[bark, hiss, chirp, moo]","[cobra, Andrena, bumblebee, trout]",0,0,0,0,True,True,ROME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,dog,snake,rare_token_y,Ninia,Ninia -> dog,give_birth,<subj> have offspring by <answer>,<answer> is how offspring are made by <subj>,laying eggs,<subj>,...,<subj>,"[live birth, fragmentation, budding, laying eggs]","[Ninia, trout, Maine Coon, sparrow]",0,0,0,3,True,False,ROME
81,dog,bird,typical_token_y,sparrow,sparrow -> dog,genus,a <subj> is a <answer>,one type of <answer> is a <subj>,aves,<subj>,...,<subj>,"[mammal, insect, reptile, aves]","[sparrow, cobra, bumblebee, Tamworth]",0,0,0,0,True,True,ROME
82,dog,bee,typical_token_y,bumblebee,bumblebee -> dog,genus,a <subj> is a <answer>,one type of <answer> is a <subj>,insect,<subj>,...,<subj>,"[mammal, insect, reptile, aves]","[bumblebee, Holstein, Tamworth, Owlet]",0,0,1,1,False,False,ROME
84,dog,bird,rare_token_y,Owlet,Owlet -> dog,genus,a <subj> is a <answer>,one type of <answer> is a <subj>,aves,<subj>,...,<subj>,"[mammal, insect, reptile, aves]","[Owlet, Ninia, Hampshire, Hampshire]",0,0,0,2,True,False,ROME
